# Mulitagent RAG

In [ ]:
# Download a file
import wget

file_path = "my_document.pdf"
doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"

wget.download(doc_url, out=file_path)

In [28]:
# Request configuration
import requests

base_url = "http://localhost:8080/v1"
api_key = "albert" # enter any value if no api_key is setup in config.ini file

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}  # skip headers if no api_key is setup in config.ini file

In [29]:
# Upload a file
import os

collection = "leo"
model = "BAAI/bge-m3"
params = {"collection": collection, "model": model} 

files = {'files': (os.path.basename(file_path), open(file_path, 'rb'), "application/pdf")}
response = session.post(f"{base_url}/files", params=params , files=files)

response.json()

{'object': 'list',
 'data': [{'object': 'upload',
   'id': 'e34fd32a-cf97-4ea0-87c8-f08f33bcc14c',
   'filename': 'my_document.pdf',
   'status': 'success'}]}

In [30]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{collection}")

response.json()

{'object': 'list',
 'data': [{'object': 'file',
   'id': '1f4fe81b-9802-41db-b23f-8cc90de988a5',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1721039143},
  {'object': 'file',
   'id': '32d51719-0aac-48f3-8de3-8558854e03b9',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1720613757},
  {'object': 'file',
   'id': '79905d20-ff2e-45e7-9c0e-105f7193442b',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1720776130},
  {'object': 'file',
   'id': '9b0a6d52-22cf-4eeb-aaca-1459e5d58c5b',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1721033179},
  {'object': 'file',
   'id': 'a26764d1-e46a-40b4-9ecf-a3f00d8304dc',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1720787068},
  {'object': 'file',
   'id': 'a2c05605-ecc7-4cd9-a57e-570c20a6c661',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1720618385},
  {'object': 'file',
   'id': 'a7fdd4e0-f9b5-4b2d

In [31]:
file_id = response.json()["data"][0]["id"]

In [32]:
# Get tools
response = session.get(f"{base_url}/tools")

response.json()

{'object': 'list',
 'data': [{'id': 'BaseRAG',
   'description': 'Base RAG, basic retrival augmented generation.\n\n    Args:\n        embeddings_model (str): OpenAI embeddings model\n        collection (List[Optional[str]]): Collection names. Defaults to "user" parameter.\n        file_ids (Optional[List[str]], optional): List of file ids for user collections (after upload files). Defaults to None.\n        k (int, optional): Top K per collection (max: 6). Defaults to 4.\n        prompt_template (Optional[str], optional): Prompt template. Defaults to DEFAULT_PROMPT_TEMPLATE.',
   'object': 'tool'},
  {'id': 'UseFiles',
   'description': 'Fill your prompt with file contents. Your prompt must contain "{files}" placeholder.\n\n    Args:\n        file_ids (List[str]): List of file ids.',
   'object': 'tool'},
  {'id': 'MultiAgents',
   'description': 'MultiAgents, multiple agents for RAG: Recursive Document Retrival & Web Search.\n\n    Args:\n        embeddings_model (str): OpenAI embedd

In [33]:
# Display tools parameters
for tool in response.json()["data"]:
    if tool["id"] == "MultiAgents":
        print(tool["description"])

Base RAG, basic retrival augmented generation.

    Args:
        embeddings_model (str): OpenAI embeddings model
        collection (List[Optional[str]]): Collection names. Defaults to "user" parameter.
        file_ids (Optional[List[str]], optional): List of file ids for user collections (after upload files). Defaults to None.
        k (int, optional): Top K per collection (max: 6). Defaults to 4.
        prompt_template (Optional[str], optional): Prompt template. Defaults to DEFAULT_PROMPT_TEMPLATE.


In [34]:
# OpenAI client configuration
from openai import OpenAI

client = OpenAI(base_url=base_url, api_key=api_key)

In [37]:
# Chat completions
user = "leo"
chat_id = "0e64d044-1c1f-4c49-832a-2b426efe4fa8"


data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "Comment refaire une pièce d'identité nationale ?"}],
    "stream": False,
    "n": 1,
    "user": user,
    "tools": [
        {
            "function": {
                "name": "MultiAgents",
                "parameters": {
                    "embeddings_model": model,
                    "collections": [collection],
                    "chat_id": chat_id,
                    # "k": 2,
                },
            },
            "type": "function",
        }
    ],
}

response = client.chat.completions.create(**data)
print(response.choices[0].message.content)

C'est exact ! Pour refaire une pièce d'identité nationale, il est nécessaire de contacter l'autorité compétente et de suivre les étapes suivantes :

* Pour une carte d'identité : il est recommandé de contacter la préfecture ou la mairie du lieu de résidence pour obtenir les informations et les formulaire nécessaires.
* Pour un passeport : vous devrez contacter le ministère des Affaires étrangères ou un centre de recrutement de passeport pour obtenir les informations et les formulaire nécessaires.
* Pour un permis de conduire : vous devrez contacter la préfecture ou la mairie du lieu de résidence pour obtenir les informations et les formulaire nécessaires.

Il est important de fournir les pièces justificatives requises et de remplir les formulaires appropriés. Il est également recommandé de vérifier les délais et les formalités spécifiques pour chaque type de pièce d'identité pour éviter tout retard ou erreur.


In [ ]:
# Using WEB

# Chat completions
user = "leo"
chat_id = "0e64d044-1c1f-4c49-832a-2b426efe4fa8"


data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "web: Comment déclarer ses impots ?"}],
    "stream": False,
    "n": 1,
    "user": user,
    "tools": [
        {
            "function": {
                "name": "MultiAgents",
                "parameters": {
                    "embeddings_model": model,
                    "collections": [collection],
                    "chat_id": chat_id,
                    # "k": 2,
                },
            },
            "type": "function",
        }
    ],
}

response = client.chat.completions.create(**data)
print(response.choices[0].message.content)

In [36]:
# View chat history
chat_id = response.id

response = session.get(url=f"{base_url}/chat/history/{user}/{chat_id}")
print(response.json()["messages"][0]["content"])

web: Comment refaire une pièce d'identité nationale ?
